

# <center> **TP2 INF4248: RNN+LSTM+Text**


---


<center> Members

 ## <center> **Hapi Kamgang Franck  ( 18T2418 )**
 ## <center> **Ngoran Aristide Fondzela ( 20V2896 )**
 ## <center> **NUNMUA SONTSA Belvanie Kartel ( 19M2319 )**

---





In [2]:
#Connextion de colab au google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Importation des bibliotheques
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [4]:
# Chargement du dataset
filename = "/content/drive/MyDrive/wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [15]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [18]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars) # nombre total de caracteres contenu dans le document
print("Total Vocab: ", n_vocab) # Nombre total de caractere distinct contenu dans le document apres convertion des caracteres en minuscules

Total Characters:  144584
Total Vocab:  49


In [19]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  144484


Les LSTM, attendent en entrée des donnees sous un format à 3 dimensions. Cela permet au modèle de traiter des séquences d'informations et de capturer les relations entre les caractères au sein d'une séquence.
En remodelant dataX dans ce format, on prépare nos donnees à être introduites dans un modèle LSTM pour l'apprentissage.

In [27]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

### **Réseau de neuronal récurrent LSTM**

### **Definir le model**

In [ ]:
# define the LSTM(Long Short-Term Memory) model

# 256: Correspond au nombre neurones cachés dans la couche LSTM.
# Ces neurones représentent la mémoire interne du réseau et permettent de capturer les dépendances à long terme entre les caractères dans les séquences textuelles.
# input_shape=(X.shape[1], X.shape[2]): Définit la forme des données d'entrée attendues par la couche LSTM.
# X.shape[1]: Correspond à la longueur des séquences d'entrée (nombre de caractères par séquence dans X).
# X.shape[2]: Correspond au nombre de caractéristiques par caractère (dans ce cas, on suppose que chaque caractère est représenté par une seule valeur, donc 1).

# 0.2: Indique le taux de désactivation (dropout rate) de 20%.
# Le dropout permet de désactiver aléatoirement 20% des neurones à chaque itération d'entraînement, ce qui aide à prévenir le surapprentissage (overfitting) du modèle.

# y.shape[1]: Correspond au nombre de neurones dans la couche dense, qui doit être égal au nombre de classes de sortie (nombre de caractères possibles dans le vocabulaire).
# activation='softmax': Définit la fonction d'activation de la couche dense qui normalise les sorties de la couche en des probabilités comprises entre 0 et 1.
# Chaque sortie représente la probabilité que le caractère suivant appartienne à une classe particulière.

# La compilation configure le modèle pour l'entraînement en spécifiant la fonction de perte, l'optimiseur et d'autres paramètres.
# loss='categorical_crossentropy': Définit la fonction de perte à minimiser pendant l'entraînement.
# optimizer='adam': Définit l'optimiseur utilisé pour mettre à jour les poids du réseau pendant l'entraînement.
# Adam est un optimiseur populaire qui ajuste efficacement les poids du réseau.

model = Sequential() # Crée un modèle séquentiel vide
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]))) # Ajoute une couche LSTM au modèle
model.add(Dropout(0.2)) # Ajoute une couche de dropout au modèle.
model.add(Dense(y.shape[1], activation='softmax')) # Ajoute une couche dense au modèle.
model.compile(loss='categorical_crossentropy', optimizer='adam') # Compile le modèle.

# Définit le chemin d'accès pour sauvegarder les poids du modèle.
# {epoch:02d}: Sera remplacé par le numéro de l'époque d'entraînement en cours (formatté sur 2 chiffres).
# {loss:.4f}: Sera remplacé par la valeur de la perte à la fin de l'époque (formatté sur 4 chiffres après la virgule).
# ".keras": Extension de fichier des poids du modèle.
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.keras"

# Crée un objet ModelCheckpoint.
# filepath: Spécifie le chemin d'accès défini précédemment.
# monitor='loss': Indique que l'on surveille la valeur de la perte pendant l'entraînement.
# verbose=1: Affiche un message d'information à chaque fois qu'un nouveau meilleur poids est sauvegardé.
# save_best_only=True: Sauvegarde uniquement les poids du modèle qui correspondent à la plus faible valeur de la perte observée jusqu'à présent.
# mode='min': Indique que l'on cherche à minimiser la valeur de la perte (on sauvegarde le modèle avec la plus faible perte).
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint] # Crée une liste contenant l'objet ModelCheckpoint.


# Entrainement du modele
# X : Ce sont les données d'entraînement, collection de séquences d'entiers (représentant des caractères) à partir desquelles le modèle apprendra..
# y : Ce sont les données cibles. Elles contiennent le caractère suivant dans la séquence pour chaque exemple d'entraînement présent dans X.
# epochs=20 : Cela spécifie le nombre d'époques d'entraînement. Une époque correspond à un passage complet sur l'ensemble des données d'entraînement.
# batch_size=128 : Ceci définit la taille du lot (batch size). La taille du lot détermine le nombre d'exemples d'entraînement traités par le modèle lors d'une seule mise à jour pendant l'entraînement.
# callbacks=callbacks_list : Cela spécifie une liste de fonctions de rappel (callback functions) qui sont invoquées pendant le processus d'entraînement. Ici, callbacks_list contient l'objet ModelCheckpoint défini précédemment.
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)


Epoch 1/20
1128/1129 [============================>.] - ETA: 0s - loss: 2.9990
Epoch 1: loss improved from inf to 2.99868, saving model to weights-improvement-01-2.9987.keras
1129/1129 [==============================] - 17s 13ms/step - loss: 2.9987
Epoch 2/20
1126/1129 [============================>.] - ETA: 0s - loss: 2.7995
Epoch 2: loss improved from 2.99868 to 2.79957, saving model to weights-improvement-02-2.7996.keras
1129/1129 [==============================] - 15s 13ms/step - loss: 2.7996
Epoch 3/20
1129/1129 [==============================] - ETA: 0s - loss: 2.6995
Epoch 3: loss improved from 2.79957 to 2.69947, saving model to weights-improvement-03-2.6995.keras
1129/1129 [==============================] - 15s 13ms/step - loss: 2.6995
Epoch 4/20
1127/1129 [============================>.] - ETA: 0s - loss: 2.6224
Epoch 4: loss improved from 2.69947 to 2.62218, saving model to weights-improvement-04-2.6222.keras
1129/1129 [==============================] - 15s 13ms/step - loss:

### **Génération de texte à l'aide du réseau LSTM formé**

In [ ]:
# load the network weights
filename = "/content/weights-improvement-20-1.9861.keras"
model.load_weights(filename)

# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Total Characters:  144584
Total Vocab:  49
Total Patterns:  144484
Seed:
" rning,” shouted the queen, stamping on the
ground as she spoke; “either you or your head must be off "
333l33333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333333

### **Réseau de neuronal récurrent LSTM plus grand**




In [ ]:
# Larger LSTM Network to Generate Text for Alice in Wonderland

# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.keras"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# fit the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Total Characters:  144584
Total Vocab:  49
Total Patterns:  144484
Epoch 1/50
2258/2258 [==============================] - ETA: 0s - loss: 2.8262
Epoch 1: loss improved from inf to 2.82625, saving model to weights-improvement-01-2.8262-bigger.keras
2258/2258 [==============================] - 46s 19ms/step - loss: 2.8262
Epoch 2/50
2256/2258 [============================>.] - ETA: 0s - loss: 2.4331
Epoch 2: loss improved from 2.82625 to 2.43285, saving model to weights-improvement-02-2.4329-bigger.keras
2258/2258 [==============================] - 41s 18ms/step - loss: 2.4329
Epoch 3/50
2256/2258 [============================>.] - ETA: 0s - loss: 2.2316
Epoch 3: loss improved from 2.43285 to 2.23151, saving model to weights-improvement-03-2.2315-bigger.keras
2258/2258 [==============================] - 42s 19ms/step - loss: 2.2315
Epoch 4/50
2257/2258 [============================>.] - ETA: 0s - loss: 2.0980
Epoch 4: loss improved from 2.23151 to 2.09802, saving model to weights-improv

### **Génération de texte à l'aide du réseau LSTM plus grand formé**

In [ ]:
# Load Larger LSTM network and generate text

# load the network weights
filename = "/content/weights-improvement-48-1.2621-bigger.keras"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Total Characters:  144584
Total Vocab:  49
Total Patterns:  144484
Seed:
" riting very busily on slates. “what are
they doing?” alice whispered to the gryphon. “they can’t hav "
e to say it out of the tame with its here, what is is?”

“i don’t know what the pabbit hirst the beginning about it,” said the cat. 
“aod that it advantar that it wastend your pardon!” said the cat. 
“and that it advantar the earth, the dould not a courle?” she said to herself, “it would be wery such a baby, i shall see the way out of the tame was oot a bit, if i can’t tell you mike them!”

“i don’t know what the pueen of the great was into a pames tay in the same with the torm!”

“i wonder what i say that is as all?” said the mock turtle. 
“nf course they dres the way it is a poot as the same thing to be a louse the season in the sea. “i don’t think the words don’t be none to sea to as the same with the toiak? but i co sert like them that _shat_ iis seall! the doumta way in my time?”

“i don’t know what the pa